### Let's go on a text adventure!

In [3]:
%run ta/story.py

THE MAGIC MISSION

castle tower
You wake up on the cold, stone floor. A rectangle of orange light strike the
floor. You see a man dressed black trench coat. He says to you, "the wizard
request you to go on a quest. A book you will take or maybe just make. A shell
which will wisper and tell. A cloth as big as a blanket. Everything else you
will find on your way."

A rectangle of light.

North: Hillside Street
East: Harbor Street

>>> North
I do not understand that command. Type "help" for a list of commands.

>>> north
You move to the north.
Hillside Street
The beautiful statue of Ben Franklin with flowers around the statue of Ben
Franklin. A women is selling flowers

A bunch of flowers wrapped in a ribbon.

South: castle tower
East: Bakery
West: Hillside Park

>>> take flowers
You take a bunch of flowers.

>>> e
You move to the east.
Bakery
The delightful smell of meat pies fills the air, making you hungry. The baker
flashes a grin, as he slides a box marked `Not Human Organs` under a 

KeyError: u'drift would'

This game was created by a 9 yr-old, so it may be impossible to solve. But one thing you can be sure of, it can be implemented with a Deterministic Finite State Machine. The only data required to hold on the information in this world is contained in a [json file](the-magic-mission.json) of connections between states and the descriptions of those states.
These sorts of games were popular in the 70's and 80's when AI and PCs were just taking off.
As we look back, we realize that these were some of the first natural language conversational dialog engines.
Mathematicians quickly came up with a data structure and model of the processing required to implement these "worlds" -- regular expressions, one of the most restricted forms of a Turing Machine. You can prove a lot of things and employ a lot of algoebra with regexes that you can't with a general Turing Machine or less restrictive  If you think about each statement you input into the console as a "symbol" and each new location in the world as a "state" then the connection to Deterministic Finite State Automata (Regular Expressions) becomes clear.
But what about complex worlds where you can do more than just move around, like pick up things required to make future actions possible. That's called a context-sensitive finite state machine. The automata is given a FIFO stack to use to store and retreive one thing from the top of a stack of things that can be "remembered." Things get a lot more complicated if you want to "solve" one of these games automatically or enumerate all their possible states. Though the states are enumerable it may not be impractical to explore them all.

## So let's use regexes to make this world less like a regex!

In [ ]:
import re

with open('ta/story.py', 'rb+') as f:
    s = re.sub(r'state\[\s*(["\'a-zA-Z_ ]+)\s*\]\[\s*(["\'a-zA-Z_ ]+)\s*\]', r'fuzzy_get(fuzzy_get(state, \1), \2)', f.read(), re.M)
    s = re.sub(r'state\[\s*(["\'a-zA-Z_ ]+)\s*\], r'fuzzy_get(state, \1)', s, re.M)
    f.seek(0)
    f.write(s)

# !sed -i 's/fuzzy_get\(state,\ (["\'a-zA-Z ]+)\)\[\s*(["\'a-zA-Z ]+)\s*\]/fuzzy_get(fuzzy_get(state, \1), \2)/g' ta/story.py

## But `fuzzy_get` (`fuzzywuzzy`) doesn't use regexes it uses `str` similarity

In [1]:
'is equality testing' == 'the best we can do'

False

In [7]:
from __future__ import division
s1, s2 = 'we can count shared words and characters', 'as a fraction of the total words \'n chars in both strs'
print(len(set(s1).intersection(set(s2))) / len(set(s1).union(set(s2))))
w1, w2 = s1.split(' '), s2.split(' ')
print(len(set(w1).intersection(set(w2))) / len(set(w1).union(set(s2))))


0.631578947368
0.04


In [15]:
s1, s2 = 'but what about repeated words words words and characters', 'when you reuse words words words and chars within each string'
print(len(set(s1).intersection(set(s2))) / len(set(s1).union(set(s2))))
w1, w2 = s1.split(' '), s2.split(' ')
print(len(set(w1).intersection(set(w2))) / len(set(w1).union(set(s2))))

0.722222222222
0.0869565217391
[[ 6  8  2  2  4  5  0  0  2  4  1  1  4  6  2  5  4  0]
 [ 3 10  2  0  4  4  1  3  4  4  4  0  6  6  2  2  5  1]]
7.54983443527
1/10.0-norm of difference: 2.4183996106e+11
1/2.0-norm of difference: 309.234246567
1/1.0-norm of difference: 25.0
1/0.5-norm of difference: 7.54983443527


## Why not create a histogram of the counts of `char`s and words?

In [ ]:
from collections import Counter
c1, c2 = Counter(list(s1)), Counter(list(s2))
print(c1)
print(c2)

## Then find a common set of dimensions? (scikit-learn calls this a `Vectorizer`)

In [21]:
keys = sorted(set(c1).union(set(c2)))
import numpy as np
import pandas as pd
v1, v2 = np.array([c1.get(k, 0) for k in keys]), np.array([c2.get(k, 0) for k in keys])
pd.DataFrame(np.matrix([v1,v2]), columns=keys, index=['v1', 'v2'])


,,a,b,c,d,e,g,h,i,n,o,p,r,s,t,u,w,y
v1,8,6,2,2,5,4,0,2,0,1,4,1,6,4,5,2,4,0
v2,10,3,0,2,4,4,1,4,3,4,4,0,6,6,2,2,5,1


## And use conventional vector algebra...

In [29]:
print(np.sqrt(np.sum((v1 - v2)**2)))
for p in (0.1, 0.5, 1, 2):
    # dist = (np.sum(np.abs(v1 - v2)**p))**(1./p)
    dist = norm(v1 - v2, p)
    print('1/{}-norm of difference: {}'.format(1./p, dist))
    print('normalized 1/{}-norm of difference: {}'.format(1./p, dist / max(norm(v1, p), norm(v2, p))))


7.54983443527
1/10.0-norm of difference: 2.4183996106e+11
normalized 1/10.0-norm of difference: 0.067094596788
1/2.0-norm of difference: 309.234246567
normalized 1/2.0-norm of difference: 0.343500456814
1/1.0-norm of difference: 25
normalized 1/1.0-norm of difference: 0.409836065574
1/0.5-norm of difference: 7.54983443527
normalized 1/0.5-norm of difference: 0.429495074963


In [ ]:
## Or find the cosine of the angle between these high-dimensional vectors (cosine distance)

In [28]:
from numpy.linalg import norm
cos_dist = np.dot(v1 / norm(v1), v2 / norm(v2))
print('cosine distance: {:.3f}\ncosine similarity: {:.3f}'.format(1 - cos_dist, cos_dist) )


cosine distance: 0.097
cosine similarity: 0.903
